
# Procesamiento de Lenguaje Natural

## Modelado de temas por LDA para ver el porcentaje de pertenencia de una obsrvación a una categoria


In [ ]:
#! pip install pyLDAvis
!pip install -U pandas-profiling

In [ ]:
import re
import pandas as pd 
from pprint import pprint

from nltk.corpus import stopwords
import nltk
import pyLDAvis.gensim_models
from gensim.models import LdaModel
from gensim.corpora import Dictionary

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
nltk.download('stopwords')
stopwords = stopwords.words('spanish')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
data = pd.read_csv('reviews_vidjew_es.csv')
data.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games


### Hacer pre-procesamiento del texto

In [4]:

def pre_procesado(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d]+", " ", texto)
    texto = [palabra for palabra in texto.split() if palabra not in stopwords]
    return texto 

data['Pre-Processed'] = data['review_body'].apply(lambda texto: pre_procesado(texto))

data.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category,Pre-Processed
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry,"[buen, buena, calidad, buena, presentación]"
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games,"[producto, perfecto, salir, casa, nintendo, sw..."
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games,"[funciona, nintendo, switch, forma, emparejarl..."
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games,"[recomendado, utilizo, pc, dan, ningún, proble..."
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games,"[cable, funciona, bien, podria, ser, mas, larg..."


### Modelo de LDA

In [5]:
dictionary = Dictionary(data['Pre-Processed'].values)
dictionary.filter_extremes(no_below=5, no_above=0.5) 
corpus = [dictionary.doc2bow(text) for text in data['Pre-Processed'].values]
model = LdaModel(corpus = corpus, id2word = dictionary, num_topics = 2, passes=50)

In [6]:
model.print_topics(num_words=10)

[(0,
  '0.029*"producto" + 0.018*"bien" + 0.015*"precio" + 0.015*"llegado" + 0.015*"foto" + 0.015*"puede" + 0.014*"día" + 0.013*"regalo" + 0.013*"llegó" + 0.012*"caja"'),
 (1,
  '0.029*"bien" + 0.029*"calidad" + 0.024*"juego" + 0.019*"si" + 0.015*"precio" + 0.015*"buena" + 0.012*"mando" + 0.011*"mas" + 0.010*"perfecto" + 0.009*"bonitos"')]

###  Punto 3: Visualización de LDA

In [7]:
lda_display=pyLDAvis.gensim_models.prepare(model, corpus, dictionary, sort_topics=True)
pyLDAvis.display(lda_display)

###  Pruebas

In [8]:
d = dictionary.doc2bow(["bailar", "rapido", "brillo"])
topics = [(cluster[0]+1, cluster[1]) for cluster in model.get_document_topics(d)]
topics

[(1, 0.19133723), (2, 0.8086628)]